In [1]:
import gym
import numpy as np
from collections import defaultdict

In [2]:
env = gym.make('Blackjack-v0')

[2019-02-09 17:21:32,791] Making new env: Blackjack-v0
/Users/falkvandermeirsch/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:18: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
env.action_space.n

2

In [4]:
def sample_policy(observation):
    score, dealer_score, usable_ace = observation
    return 0 if score >= 20 else 1

In [5]:
def generate_episode(policy, env):
    states, actions, rewards = [], [], []
    
    # initialize the first state
    observation = env.reset()
    
    while True:
        states.append(observation)
        
        # now we select the next action using our policy
        action = policy(observation)
        actions.append(action)
        
        # we perform the action and move to the next state
        observation, reward, done, _ = env.step(action)
        rewards.append(reward)
        
        if done:
            break
        
    return states, actions, rewards
        

MC prediction

In [6]:
def first_visit_MC_prediction(policy, env, n_episodes):
    # value_table and state counter initialization
    value_table = defaultdict(float)
    N = defaultdict(int)
    
    for i in range(n_episodes):
        # generatinge episodes
        states, actions, rewards = generate_episode(policy, env)
        returns = 0
        
        # for each step, we store the rewards to a variable R and the states to a variable S
        # we calculate returns as the sum of the rewards
        for t in range(len(states) - 1, 1, -1):
            R = rewards[t]
            S = states[t]
            
            returns += R
            
            # check if the state is visited for the first time, if so N(s) is incremented
            if S not in states[:t]:
                N[S] += 1
                value_table[S] += (returns - value_table[S]) / N[S]
    
    return value_table

In [7]:
value_table = first_visit_MC_prediction(sample_policy, env, 1_000_000)

In [8]:
value_table

defaultdict(float,
            {(17, 9, False): -0.6634512325830663,
             (20, 9, False): 0.7568160451269185,
             (14, 9, False): -0.6384615384615383,
             (21, 10, False): 0.888157894736843,
             (19, 3, False): -0.7336496103015918,
             (21, 6, False): 0.9058295964125559,
             (20, 1, False): 0.1690702669530531,
             (16, 1, False): -0.7302133516861675,
             (14, 10, False): -0.6178125000000004,
             (20, 10, False): 0.4277718141389355,
             (16, 10, False): -0.671993271656853,
             (13, 10, False): -0.6059322033898286,
             (16, 4, False): -0.6744652406417121,
             (21, 1, False): 0.6330472103004288,
             (15, 1, False): -0.6994485294117647,
             (13, 3, False): -0.5743589743589749,
             (19, 2, False): -0.743141289437586,
             (17, 10, False): -0.6980250867360547,
             (17, 10, True): -0.44354838709677424,
             (15, 10, False): -0.